In [1117]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def ColeColePelton(f, sigmaInf, eta, tau, c):
    w = 2*np.pi*f
    return sigmaInf*(1 - eta/(1 + (1-eta)*(1j*w*tau)**c))

def ColeColeSeigel(f, sigmaInf, eta, tau, c):
    w = 2*np.pi*f
    return sigmaInf*(1 - eta/(1 + (1j*w*tau)**c))

In [1118]:
taumin, taumax = 1e-10, 1e8
ntau = 71
mesh = getTau(taumin, taumax, ntau)
frequency = np.logspace(-6, 8, 61)

In [1107]:
from SimPEG import (DataMisfit, Regularization, Inversion,
                    Optimization, InvProblem)

mesh = getFrequency(fmin, fmax, nfreq)
wires = Maps.Wires(('sigmaInf', 1), ('eta', mesh.nN))
sigmaInf, tau, c = 0.01, 0.1, 1
sigmap = Maps.ExpMap(nP=1)*wires.sigmaInf
# sigmap = wires.sigmaInf
prb = DebyeDec(mesh, sigmaInfMap = sigmap, etaMap = wires.eta, frequency=frequency)
survey = DebyeDecSurvey()
prb.pair(survey)
eta = np.zeros(prb.ntau) +  0.01
# eta[20] = 0.1
m0 = np.r_[np.log(sigmaInf), eta]
f0 = prb.fields(m0)
d0 = survey.dpred(m0)
# survey.makeSyntheticData(m0, std = 0.)
survey.dobs = d0
survey.std = 0.0
survey.eps = 1.
regmesh = Mesh.TensorMesh([m0.size])
# Now set up the problem to do some minimization
dmis = DataMisfit.l2_DataMisfit(survey)
reg = Regularization.BaseRegularization(regmesh)
opt = Optimization.InexactGaussNewton(
    maxIterLS=20, maxIter=10, tolF=1e-6,
    tolX=1e-6, tolG=1e-6, maxIterCG=6
)
invProb = InvProblem.BaseInvProblem(dmis, reg, opt, beta=1e4)
inv = Inversion.BaseInversion(invProb)

In [1108]:
print (1./f0).real - 1./abs(f0)**2 * f0.real
print (1./f0).imag + 1./abs(f0)**2 * f0.imag


[ -2.84217094e-14   0.00000000e+00   2.84217094e-14   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.84217094e-14
   0.00000000e+00   0.00000000e+00   2.84217094e-14   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.42108547e-14
   1.42108547e-14  -2.84217094e-14   1.42108547e-14  -1.42108547e-14
   0.00000000e+00   2.84217094e-14  -1.42108547e-14   2.84217094e-14
  -1.42108547e-14  -1.42108547e-14   2.84217094e-14   0.00000000e+00
   2.84217094e-14   1.42108547e-14  -1.42108547e-14   1.42108547e-14
   1.42108547e-14   1.42108547e-14   0.00000000e+00  -1.42108547e-14
   1.42108547e-14  -1.42108547e-14   0.00000000e+00  -2.84217094e-14
  -1.42108547e-14  -1.42108547e-14   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.42108547e-14   0.00000000e+00  -1.42108547e-14
   0.00000000e+00  -2.84217094e-14   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00

In [1109]:
# def dmis_sig(m):  
#     fpre = prb.fields(m)
#     mis = 0.5*np.linalg.norm(fpre - f0)**2
#     r = fpre - f0
#     dmis = prb.dsig_dm(r, adjoint=True)
#     return mis,dmis

# passed = Tests.checkDerivative(dmis_sig,
#     m0*4,
#     plotIt=False,
#     num=5, 
#     dx = m0*2
# )

In [1110]:
dm = np.r_[np.log(sigmaInf)-2, eta*8]
derChk = lambda m: [survey.dpred(m), lambda mx: prb.Jvec(m0, mx, f=f0)]
passed = Tests.checkDerivative(derChk, m0, plotIt=False, dx=dm, num=2)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    6.149e+00     3.416e+00      nan
 1   1.00e-02    5.983e-01     5.241e-02      1.814
========================= PASS! =========================
Well done Sgkang!



In [1111]:
passed = Tests.checkDerivative(lambda m: [dmis.eval(m), dmis.evalDeriv(m)],
    m0*1.5,
    plotIt=False,
    num=4, dx=m0*2)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.650e+02     2.784e+01      nan
 1   1.00e-02    1.399e+01     2.788e-01      1.999
 2   1.00e-03    1.374e+00     2.921e-03      1.980
 3   1.00e-04    1.372e-01     4.261e-05      1.836
========================= PASS! =========================
Testing is important.



In [1112]:
def dmis_eta(m):  
    fpre = prb.fields(np.r_[np.log(sigmaInf), m])
    r = fpre - f0
    mis = 0.5*np.dot(r.conj(), r)
    dmis = prb.dsig_deta(r, adjoint=True)
    return mis, dmis

passed = Tests.checkDerivative(dmis_eta,
    prb.eta*3,
    plotIt=False,
    num=5, 
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.529e-03     4.545e-04      nan
 1   1.00e-02    1.116e-04     4.188e-06      2.036
 2   1.00e-03    1.078e-05     4.155e-08      2.003
 3   1.00e-04    1.074e-06     4.151e-10      2.000
 4   1.00e-05    1.074e-07     4.151e-12      2.000
========================= PASS! =========================
Once upon a time, a happy little test passed.



In [1113]:
# mini = m0*2
# prb.fields(m=mini)
# out = dmis.evalDeriv(mini)
# fd = np.zeros_like(out)
# for  i, m_temp in enumerate(mini):
#     mtempa = mini.copy()
#     dm = mini[i]*0.01
#     mtempa[i] = mini[i] + dm
#     mtempb = mini.copy()
#     mtempb[i] = mini[i] - dm   
#     fd[i] = (dmis.eval(mtempa) - dmis.eval(mtempb)) / (2*dm)
# print out[0], fd[0]   
# plt.plot(out, 'ro')
# plt.plot(fd, 'k.')

In [1114]:
v = np.random.rand(regmesh.nC)
w = np.random.rand(survey.dobs.shape[0])
wtJv = w.dot(prb.Jvec(m0, v))
vtJtw = v.dot(prb.Jtvec(m0, w))
err = np.abs(wtJv - vtJtw) / np.abs(wtJv)
print err

2.15374833009e-16


In [1115]:
v = np.random.rand(regmesh.nC)
w = np.random.rand(prb.nfreq)
wtJv = (w.conj().dot(prb.dsig_dm(v))).real
vtJtw = v.conj().dot(prb.dsig_dm(w, adjoint=True))
err = np.abs(wtJv - vtJtw) / np.abs(wtJv)
print err

1.26570382004e-16


In [1116]:
v = np.random.rand(prb.nfreq)
w = np.random.rand(prb.nfreq*2)
f0 = prb.fields(m0)
wtJv = (w.conj().dot(survey.evalDeriv(v, f=f0)))
vtJtw = v.conj().dot(survey.evalDeriv(w, f=f0, adjoint=True).real) 
print wtJv
print vtJtw
err = np.abs(wtJv - vtJtw) / np.abs(wtJv)
print err

1567.09030297
1567.09030297
2.90185798499e-16


In [1071]:
# def dmis_P(m):  
#     dpre = survey.eval(m) 
#     mis = 0.5*np.linalg.norm(dpre-d0)**2
#     r = dpre - d0
#     dmis = survey.evalDeriv(r, m, adjoint=True)
#     return mis,dmis

# passed = Tests.checkDerivative(dmis_P,
#     f0*2,
#     plotIt=False,
#     num=2
# )

In [1072]:
# def dmis_siginf(m):  
#     fpre = prb.fields(np.r_[np.log(m), prb.eta])
# #     mis = 0.5*np.linalg.norm(fpre - f0)**2
#     r = fpre - f0
#     mis = 0.5*np.dot(r.conj(), r)

#     dmis = prb.dsig_dsigmaInf(r, adjoint=True)
#     return mis, dmis

# passed = Tests.checkDerivative(dmis_siginf,
#     np.r_[sigmaInf]*2,
#     plotIt=False,
#     num=4, 
#     dx = np.r_[sigmaInf]*5,
# )

In [1073]:
# def dsig_deta():
#     dsig_deta = -sigmaInf*prb.X        
#     dsig_deta -= Utils.sdiag(prb.sigmaInf*1j*prb.omega)*prb.X**2*Utils.sdiag(prb.tau*prb.eta) 
#     return dsig_deta

In [1074]:
# f0 = prb.fields(m0)
# J = dsig_deta()
# fpre = prb.fields(np.r_[np.log(sigmaInf), prb.eta*2])
# r = fpre - f0

In [1075]:
# I*eta*omega*tau/(I*omega*tau*(-eta + 1) + 1)**2 + 1/(I*omega*tau*(-eta + 1) + 1)

In [1076]:
# def foo(m):    
#     sigma = sigmaInf - sigmaInf*m/(1.+(1.-m)*(1j*omega*tau))
#     return sigma

# def Jvec(m, mx):
#     dsig_deta = -sigmaInf/(1.+(1.-m)*(1j*omega*tau))
#     dsig_deta +=  -sigmaInf*1j*omega*tau*m/(1.+(1.-m)*(1j*omega*tau))**2
#     Jv = np.r_[dsig_deta]*mx
#     return Jv


In [1077]:
def dsigdeta(m, tau, omega):
    dsig_deta = -sigmaInf/(1.+(1.-m)*(1j*omega*tau))
    dsig_deta +=  -sigmaInf*1j*omega*tau*m/(1.+(1.-m)*(1j*omega*tau))**2
    return dsig_deta

In [1079]:
temp = np.r_[np.log(sigmaInf), eta]
m0 = prb.fields(temp)

def foo_P(m):    
    out = survey.eval(m)
    return out

def Jvec_P(m, mx):
    out = survey.evalDeriv(mx, f=m)
    return out

derChk = lambda m: [foo_P(m), lambda mx: Jvec_P(m0, mx)]
passed = Tests.checkDerivative(derChk, m0, plotIt=False)

# def foo_sig(m):    
#     sigma = prb.fields(m)
#     return sigma

# def Jvec_sig(m, mx):
#     sigma = prb.fields(m)
#     Jv = prb.dsig_dm(mx)
#     return Jv

# m0 = np.r_[np.log(sigmaInf), eta]
# derChk = lambda m: [foo_sig(m), lambda mx: Jvec_sig(m0, mx)]
# passed = Tests.checkDerivative(derChk, m0, plotIt=False)


==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.634e+01     6.867e+01      nan
 1   1.00e-02    9.333e+00     8.737e+00      0.895
 2   1.00e-03    6.642e-01     8.562e-02      2.009
 3   1.00e-04    6.825e-02     4.984e-03      1.235
 4   1.00e-05    6.855e-03     4.963e-04      1.002
 5   1.00e-06    6.859e-04     4.967e-05      1.000
 6   1.00e-07    6.859e-05     4.967e-06      1.000
*********************************************************
<<<<<<<<<<<<<<<<<<<<<<<<< FAIL! >>>>>>>>>>>>>>>>>>>>>>>>>
*********************************************************
You had so much promise Sgkang, oh well...



In [1003]:
# def foo(m):    
#     sigma = prb.fields(np.r_[np.log(sigmaInf), m])
#     return sigma

# def Jvec(m, mx):
#     sigma = prb.fields(np.r_[np.log(sigmaInf), m])
#     Jv = prb.dsig_deta(mx)
#     return Jv

# def foo(m):    
#     sigma = prb.fields(np.r_[np.log(m), prb.eta])
#     return sigma

# def Jvec(m, mx):
#     sigma = prb.fields(np.r_[np.log(m), prb.eta])
#     Jv = prb.dsig_dsigmaInf(mx)
#     return Jv

# def foo(m):    
#     omega = prb.omega.copy()
#     sigma = sigmaInf - sigmaInf*m/(1.+(1.-m)*(1j*omega*tau))
#     return sigma

# def Jvec(m, mx):
#     omega= prb.omega.copy()
#     dsig_deta = -sigmaInf/(1.+(1.-m)*(1j*omega*tau))
#     dsig_deta +=  -sigmaInf*1j*omega*tau*m/(1.+(1.-m)*(1j*omega*tau))**2
#     Jv = np.r_[dsig_deta]*mx
#     return Jv


# def foo(m):    
#     sigma = sigmaInf*np.ones(prb.nfreq, dtype=complex)    
#     omega = prb.omega    
#     for itau in range(prb.ntau):
#         sigma -= sigmaInf*m[itau]/(1.+(1.-m[itau])*(1j*omega*prb.tau[itau]))
#     return sigma

# def Jvec(m, mx):
#     J = np.ones((prb.nfreq, prb.ntau), dtype=complex)
#     temp = 1+0*1j
#     omega = prb.omega
#     for itau in range(prb.ntau):
#         J[:,itau] = -sigmaInf/(1.+(1.-m[itau])*(1j*omega*prb.tau[itau]))
#         J[:,itau] +=  -sigmaInf*1j*omega*prb.tau[itau]*m[itau]/(1.+(1.-m[itau])*(1j*omega*prb.tau[itau]))**2
#     Jv = np.dot(J, mx)
#     return Jv

# m0 = prb.sigmaInf
# m0 = prb.eta
# m0 = np.r_[0.1]


In [583]:
# from sympy import *
# eta, omega, tau = symbols("eta omega tau")
# f = eta / (1+(1-eta)*I*omega*tau)
# diff(f, eta)
# f = np.logspace(-3, 3, 61)
# sigma = ColeColePelton(f, 1., 0.1, 0.1, 0.5)
# from SimPEG import Tests
# def foo(m):
#     return np.r_[np.log(m).real, np.log(m).imag]

# def Jvec(m, mx):
#     rho = 1./m
#     Jv = np.r_[rho.real*mx, rho.imag*mx ]
#     return Jv
# m0 = sigma.copy()
# derChk = lambda m: [foo(m), lambda mx: Jvec(m0, mx)]
# passed = Tests.checkDerivative(derChk, m0, plotIt=False)

In [584]:
# mx = np.zeros_like(prb.eta)
# mx[20] = 1.
# out = Jvec(prb.eta, mx)
# plt.plot(dsigdeta(prb.eta[20], prb.tau[20], prb.omega).real)
# plt.plot(out.real, 'k.')

In [585]:
# sigma_prb = prb.fields()
# plt.semilogx(prb.f, np.log(sigma_prb).real, 'k.-')
# plt.semilogx(prb.f, np.log(sigma_prb).imag, 'r.-')
# plt.grid(True)
# ymin, ymax = plt.gca().get_ylim()
# plt.plot(np.ones(2)*1./(2*np.pi*tau), np.r_[ymin, ymax], 'k-')
# plt.ylim(ymin, ymax)

In [586]:
# sigma = ColeColePelton(f, sigmaInf, 0.1, tau, c)
# etatilde = (sigma-sigmaInf)/sigmaInf

In [587]:
# np.linalg.norm(sigma_prb-sigma)

In [588]:
# plt.semilogx(f, np.log(sigma/sigmaInf).real, 'k.-')
# plt.semilogx(f, np.log(sigma).imag, 'r.-')
# plt.grid(True)
# ymin, ymax = plt.gca().get_ylim()
# plt.plot(np.ones(2)*1./(2*np.pi*tau), np.r_[ymin, ymax], 'k-')
# plt.ylim(ymin, ymax)

In [589]:
340./2500 * 1e3

136.0

In [590]:
21000./ 2000.

10.5

In [591]:
30*1e-3 / 5

0.006

In [592]:
1./50

0.02